This class breaks up the problem into 3 main sections:
- ```match_data```: matching the linescans to polygons to create training data
- ```error_fraction```: calculating the quality of a solution
- ```mask```: function to determine whether a coordinate is on fire

### Example:

In [2]:
%load_ext autoreload
%autoreload 2

from solution import Threshold

/env/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
# Simple threshold masking
s = Threshold()

# Example: set the threshold to an arbitrary value
s.set_threshold(220) 

# Set kernel sizes
s.set_open_kernel(1)
s.set_close_kernel(64)

# Match training data
s.match_data()

# Get error
print('Error:', s.error_fraction())

### Simple optimisation routine
Try a range of thresholds and find the best one

In [ ]:
error = 1.0

s = Threshold()
s.match_data()

# Set all kernels=1
s.set_open_kernel(1)
s.set_close_kernel(1)

print()

for t in range(180, 220, 5):
    s.set_threshold(t)
    e = s.error_fraction()

    if e < error:
        error = e

        print('Best solution so far:')
        print(' threshold    = ', t)
        print(' error        = ', error)
        print()

### SciPy Optimize
Optimise for multiple variables

In [3]:
s = Threshold()
s.match_data()

134 linescan datasets available
129 training linescan datasets available
37 linescans with matching polygons
28.7% of linescans used
45.0% of polygons used


In [13]:
from scipy import optimize
import numpy as np

def f(params):
    t, kc = params

    s.set_threshold(t)
    s.set_close_kernel(kc)
    error = s.error_fraction()
    
    print('{0: 3.2f}   {1: 3.2f}   {2: 3.4e}'.format(t, kc, error))

    return error
    
initial_guess = [195, 64]
initial_simplex = np.zeros((3, 2))
initial_simplex[0,:] = [180, 1]
initial_simplex[1,:] = [180, 100]
initial_simplex[2,:] = [220, 100]

result = optimize.minimize(f, initial_guess, method='Nelder-Mead', options={'disp': True, 'initial_simplex': initial_simplex})

fitted_params = result.x
print(fitted_params)
    

 180.00    1.00    5.7249e-02
 180.00    100.00    5.6649e-02
 220.00    100.00    5.6538e-02
 220.00    199.00    5.8437e-02
 190.00    50.50    5.6451e-02
 230.00    50.50    5.6722e-02
 192.50    87.62    5.6259e-02
 162.50    38.12    5.6608e-02
 205.62    84.53    5.6353e-02
 208.12    121.66    5.6638e-02
 194.53    68.29    5.6357e-02
 203.59    103.87    5.6354e-02
 201.33    94.97    5.6326e-02
 188.20    98.07    5.6288e-02
 179.38    90.72    5.6345e-02
 195.84    93.91    5.6127e-02
 200.14    83.47    5.6252e-02
 203.48    89.75    5.6294e-02
 195.24    88.16    5.6216e-02
 190.95    98.60    5.6238e-02
 193.24    94.82    5.6157e-02
 193.84    100.57    5.6222e-02
 194.89    91.26    5.6186e-02
 194.19    97.47    5.6151e-02
 196.79    96.56    5.6153e-02
 195.90    96.12    5.6152e-02
 194.13    95.25    5.6126e-02
 193.24    94.82    5.6157e-02
 195.78    91.70    5.6180e-02
 194.59    96.02    5.6161e-02
 194.98    94.58    5.6156e-02
 194.16    96.36    5.6161e-02
 19

### Save the submission file

In [15]:
# s.set_threshold(194)
# s.set_close_kernel(95)
# s.error_fraction()
s.generate_submission(f'submission_threshold_194_kc_95.csv')